In [ ]:
#yolov8n.pt
#yolov8s.pt
#yolov8m.pt
#yolov8l.pt
#yolov8x.pt

In [1]:
import cv2
from ultralytics import YOLO

In [2]:
#----------------------------------Cargando el modelo nano y los videos
model = YOLO('yolov8n.pt')
video_path = '/Users/zulybercampos/Documents/Flowsense/videos/Video1.mp4'
cap = cv2.VideoCapture(video_path)

In [ ]:
import pandas as pd
from datetime import timedelta

#------------- Capturando las dimensiones del video
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

# La linea vertical estará al 80%
line_x = int(frame_width * 0.80)

#----------------- Variables como conteo de personas, tracker, posición de la persona
person_count = 0
tracked_ids = set()
last_positions = {}

# DataFrame para guardar entradas
entradas_df = pd.DataFrame(columns=["ID", "Tiempo_seg", "Tiempo_hhmmss"])

# -----------------------iniciando las iteraciones
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(frame, persist=True)

    # Dibujar línea vertical (entrada a la derecha)
    cv2.line(frame, (line_x, 150), (line_x, frame_height-50), (0, 255, 0), 3)

    if results[0].boxes is not None:
        for box in results[0].boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            if cls == 0 and conf >= 0.6:
                # -------------------- Coordenadas de la caja
                x1, y1, x2, y2 = box.xyxy[0]
                cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)  # centro

                # -------------------- ID único del tracker
                track_id = int(box.id[0]) if box.id is not None else None

                # -------------------- Dibujar bounding box
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
                cv2.putText(frame, f"ID {track_id} {conf:.2f}", (int(x1), int(y1) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

                # -------------------- Dibujar centro
                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

                # ------------------------------ Verificar dirección del cruce
                if track_id is not None:
                    last_cx = last_positions.get(track_id, None)

                    # Estableciendo el conteo para que venga de la derecha
                    if last_cx is not None and last_cx > line_x and cx < line_x:
                        if track_id not in tracked_ids:
                            person_count += 1
                            tracked_ids.add(track_id)

                            # --- Guardar evento en DataFrame
                            tiempo_seg = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
                            tiempo_hhmmss = str(timedelta(seconds=int(tiempo_seg)))
                            entradas_df.loc[len(entradas_df)] = [track_id, tiempo_seg, tiempo_hhmmss]

                    # -------------------------última posición
                    last_positions[track_id] = cx

    # ------------------------Mostrar contador en pantalla
    cv2.putText(frame, f"Entradas: {person_count}", (20, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 2)

    # -------------------------------Mostrar frame
    cv2.imshow("Detección y Tracking", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):   # salir
        break
    elif key == ord('p'): # pausar
        print("⏸️ Video en pausa. Presiona 'p' para continuar...")
        while True:
            pause_key = cv2.waitKey(0) & 0xFF
            if pause_key == ord('p'):  # reanudar solo con 'p'
                print("▶️ Reanudando video...")
                break
            elif pause_key == ord('q'):  # salir desde pausa
                cap.release()
                cv2.destroyAllWindows()
                exit()

# -------------------- Se guardan los datos
cap.release()
cv2.destroyAllWindows()
entradas_df.to_csv("entradas.csv", index=False)
print("Datos guardados en entradas.csv")

In [ ]:
#cap.release()           
#cv2.destroyAllWindows() 
exit() 

: 